# Stochastic Classification of Economics Departments by Placement Rates

James Yu, 27 November 2021

In [1]:
# import Pkg
# Pkg.add("Distributions")
# Pkg.add("JSON")
using Distributions, JSON

This notebook contains an algorithm for classifying Economics departments into a series of types based on placement outcome data.

The placement data used is grouped as a dictionary of applicants sorted by year of starting application. In total, the data spans from 2003 to 2021. The procedure for compiling the data is provided in a separate Python file.

In [2]:
year_by_year = JSON.parsefile("to_from_by_year.json")
sort(collect(keys(year_by_year)))

19-element Vector{String}:
 "2003"
 "2004"
 "2005"
 "2006"
 "2007"
 "2008"
 "2009"
 "2010"
 "2011"
 "2012"
 "2013"
 "2014"
 "2015"
 "2016"
 "2017"
 "2018"
 "2019"
 "2020"
 "2021"

For our estimation, we will only consider placements from 2015 to 2018 as the number of reported graduates on EconJobMarket was the most stable and even over this period. We put all placement outcomes for Assistant Professor positions in one set, and all other placement outcomes in another set. The other placement outcomes are those where applicants were hired at departments that do not graduate Ph.D. students; this includes the public sector, private sector, government, non-academic positions at academic departments and teaching universities, among others.

In [3]:
i = 0

academic_builder = Set{}()
sink_builder = Set{}()
name_to_oid = Dict{}()

for key in 2015:2020
    data = year_by_year[string(key)]
    println(key, " has ", length(keys(data)), " entries")
    for aid in keys(data)
        outcome = data[aid]
        name_to_oid[outcome["to_name"]] = string(outcome["to_oid"])
        name_to_oid[outcome["from_institution_name"]] = string(outcome["from_oid"])
        if outcome["position_name"] == "Assistant Professor"
            push!(academic_builder, outcome)
        else
            push!(sink_builder, outcome)
        end
    end
end
println(length(academic_builder), " total assistant professor outcomes")
println(length(sink_builder), " other outcomes")

2015 has 512 entries
2016 has 1145 entries
2017 has 837 entries
2018 has 912 entries
2019 has 1002 entries
2020 has 847 entries
3608 total assistant professor outcomes
1647 other outcomes


This piece of code deals with teaching universities by checking if they ever graduated Ph.D. students:

In [4]:
academic = Set{}()
academic_to = Set{}()
for outcome in academic_builder
    push!(academic, outcome["from_institution_name"])
    push!(academic_to, outcome["to_name"])
end

tch_sink = Set{}() # sink of teaching universities that do not graduate PhDs
for key in academic_to
    if !(key in academic)
        push!(tch_sink, key)
    end
end

println(length(academic))
println(length(academic_to))
println(length(tch_sink))

361
801
494


The next piece of code sorts all the sink departments (except teaching universities, which are dealt with above) by category:

In [5]:
filter_sink_builder = Set{}()
for outcome in sink_builder
    if outcome["from_institution_name"] in academic
        push!(filter_sink_builder, outcome)
    end
end

acd_sink = Set{}()
gov_sink = Set{}()
pri_sink = Set{}()

for outcome in filter_sink_builder
    # CODE global academic, other_placements, pri_sink, gov_sink, acd_sink
    if outcome["recruiter_type"] in [6, 7]
        # private sector: for and not for profit
        push!(pri_sink, string(outcome["to_name"], " (private sector)"))
    elseif outcome["recruiter_type"] == 5
        # government institution
        push!(gov_sink, string(outcome["to_name"], " (public sector)"))
    else
        # everything else including terminal academic positions
        push!(acd_sink, string(outcome["to_name"], " (academic sink)"))
    end
end

println(length(acd_sink))
println(length(gov_sink))
println(length(pri_sink))

205
91
102


Now that we have five sets for every category of department, we can construct a matrix representing the placements between these departments:

In [6]:
institutions = vcat(collect(academic), collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink))

out = zeros(UInt8, length(institutions), length(collect(academic)))
i = 0
for outcome in academic_builder
    i += 1
    out[findfirst(isequal(outcome["to_name"]), institutions), findfirst(isequal(outcome["from_institution_name"]), institutions)] += 1
end
for outcome in filter_sink_builder
    i += 1
    keycheck = ""
    if outcome["recruiter_type"] in [6, 7]
        keycheck = string(outcome["to_name"], " (private sector)")
    elseif outcome["recruiter_type"] == 5
        keycheck = string(outcome["to_name"], " (public sector)")
    else
        keycheck = string(outcome["to_name"], " (academic sink)")
    end
    out[findfirst(isequal(keycheck), institutions), findfirst(isequal(outcome["from_institution_name"]), institutions)] += 1
end
println("Total number of outcomes: ", i)

Total number of outcomes: 5156


Finally, we get to the estimator. For this estimate, we assume that each observed set of placement outcomes between any two pairs of departments is drawn from a distribution common to the "type" of the hiring department and the "type" of the graduating department. Here this distribution is assumed to be Poisson, in line with classical stochastic block models used for similar estimations in Karrer and Newman (2011) and Peixoto (2014).

Given a particular assignment of departments to types, and given the placement outcomes, a single round of estimation computes the mean number of applicants from any single type $t$ department that would be hired at a single type $t^\prime$ department and measures the probability that each independent observation was drawn from its corresponding mean. When summed together, the logarithms of the probabilities form a log-likelihood which can be used for maximum likelihood estimation.

In [7]:
function bucket_estimator(assign::Array{UInt8}, A::Matrix{UInt8}, num::UInt8)
    b = zeros(UInt8, size(A)[1], size(A)[2])
    T = zeros(num + 1, num)
    count = zeros(num + 1, num)
    
    for i in 1:size(A)[1], j in 1:size(A)[2]
         @inbounds val = (num + 1) * (assign[j] - 1) + assign[i]
         @inbounds b[i, j] = val
         @inbounds T[val] = ((T[val] * count[val]) + A[i, j]) / (count[val] + 1)
         @inbounds count[val] += 1
    end
    L = 0.0
    for i in eachindex(A)
        @inbounds L += logpdf(Poisson(max(T[b[i]], 0)), A[i])
    end
    return -L, T
end

bucket_estimator (generic function with 1 method)

Finally, we compute the maximum-likelihood estimated Poisson means by stochastically re-allocating departments to types and saving likelihood-improving re-allocations until no further re-allocations are found.

In [8]:
function doit(sample, academic_institutions, asink, gsink, psink, tsink, all_institutions, num)
    # some initial states
    current_allocation = Array{UInt8}(undef, length(all_institutions))
    cur_objective = Inf
    best_mat = nothing
    cursor = 1
    for inst in academic_institutions
        current_allocation[cursor] = 1
        cursor += 1
    end
    # except that the sinks must stay in types
    # this was built to support more sinks, but here we only use one
    for key in asink
        current_allocation[cursor] = num + 1
        cursor += 1
    end
    for key in gsink
        current_allocation[cursor] = num + 1
        cursor += 1
    end
    for key in psink
        current_allocation[cursor] = num + 1
        cursor += 1
    end
    for key in tsink
        current_allocation[cursor] = num + 1
        cursor += 1
    end
    blankcount = 0
    
    # BEGIN MONTE CARLO REALLOCATION ROUTINE
    while true
        # attempt to reallocate academic institutions to a random spot
        temp_allocation = copy(current_allocation)
        k = rand(1:length(academic_institutions))
        @inbounds temp_allocation[k] = rand(delete!(Set(1:num), temp_allocation[k]))
        # check if the new assignment is better
        test_objective, estimated_means = bucket_estimator(temp_allocation, sample, num)
        if test_objective < cur_objective
            print(test_objective, " ")
            blankcount = 0
            cur_objective = test_objective
            best_mat = estimated_means
            current_allocation = temp_allocation
        else
            blankcount += 1
            if blankcount % 1000 == 0
                print(blankcount, " ")
            end
        end
        if blankcount == 100000
            return cur_objective, best_mat, current_allocation
        end
    end
end

SELECT_COUNT = 4 # four type allocation used here
est_obj, est_mat, est_alloc = doit(out, collect(academic), collect(acd_sink), collect(gov_sink), collect(pri_sink), collect(tch_sink), institutions, UInt8(SELECT_COUNT))

28602.100477941112 28588.49240571065 28583.92587652919 28571.798771872916 28566.27120759026 28560.231838914173 28551.450381066526 28549.01454840963 28538.96697151123 28529.93232969606 28528.13089107632 28517.638787469165 28505.046515740676 28504.477398884836 28495.546455940632 28488.302072188933 28476.029455803637 28474.493309742804 28463.986411015703 28454.624832385598 28441.781498922224 28427.97844223123 28426.685583227943 28414.936693889646 28400.797752288425 28395.767856659764 28382.24764839557 28368.242055863655 28361.12610911076 28329.783181095077 28329.677268636555 28322.276938692834 28318.785681340043 28305.87453581294 28296.924541643475 28283.684757461382 28278.00476646183 28266.95422060946 28258.56215047416 28258.133513208293 28247.927493923118 28236.29793144306 28235.993680393483 28234.59506160915 28221.588387353873 28218.516948850698 28204.90473010004 28195.067715250312 28182.44133760898 28176.677771835843 28166.437966961712 28159.321809987014 28150.405378692012 28143.77942

(23041.048203228795, [0.07511569953720185 0.0024826216484607764 0.007686932215234123 0.3108715184186885; 0.009433962264150933 0.0023372576177285464 0.0023391812865497037 0.015977443609022556; … ; 0.07097933513027847 0.002819548872180451 0.005291005291005287 0.7551020408163263; 0.019396734072256564 0.001320214774604679 0.004849692742069441 0.05573350416399737], UInt8[0x04, 0x02, 0x03, 0x01, 0x03, 0x03, 0x02, 0x01, 0x03, 0x02  …  0x05, 0x05, 0x05, 0x05, 0x05, 0x05, 0x05, 0x05, 0x05, 0x05])

In [9]:
est_mat

5×4 Matrix{Float64}:
 0.0751157   0.00248262  0.00768693  0.310872
 0.00943396  0.00233726  0.00233918  0.0159774
 0.0589797   0.00131579  0.013059    0.158025
 0.0709793   0.00281955  0.00529101  0.755102
 0.0193967   0.00132021  0.00484969  0.0557335

Test reallocating all departments:

In [14]:
new_alloc = copy(est_alloc)
for pos in 1:length(new_alloc)
    print(pos, " ")
    if new_alloc[pos] != 5
        best = new_alloc[pos]
        L, T = bucket_estimator(est_alloc, out, UInt8(4))
        for test in 1:4
            if test != new_alloc[pos]
                temp_alloc = copy(new_alloc)
                temp_alloc[pos] = test
                nL, nT = bucket_estimator(temp_alloc, out, UInt8(4))
                if nL < L
                    best = test
                    L = nL
                    T = nT
                    println("found a swap")
                elseif nL == L
                    best = test
                    L = nL
                    T = nT
                    println("found an ok swap")
                end
            end
            new_alloc[pos] = best
        end
    else
        break
    end
end

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

No higher-likelihood movements are possible.

The estimated means are:

In [16]:
function bucket_estimate_extract(assign::Array{UInt8}, A::Matrix{UInt8}, num::UInt8)
    T = zeros(Int64, num + 1, num) 
    for i in 1:size(A)[1], j in 1:size(A)[2]
         @inbounds T[(num + 1) * (assign[j] - 1) + assign[i]] += A[i, j]
    end
    return T
end

est_mat_placement = bucket_estimate_extract(est_alloc, out, UInt8(4))

5×4 Matrix{Int64}:
 211   20   55   346
  76   54   48    51
 422   27  238   448
  79    9   15   333
 917  179  584  1044

Sort into decreasing order:

In [17]:
sink_sum = sum(est_mat_placement[i, :] for i in 5:5)
indices_of_greatest_elements = sortperm(sink_sum, rev = true)
order = zeros(Int64, length(sink_sum))
for i in 1:length(sink_sum)
    order[indices_of_greatest_elements[i]] = i
end
for i in 5:5
    push!(order, i)
end
order

5-element Vector{Int64}:
 2
 4
 3
 1
 5

In [18]:
placement_rates = zeros(Int64, (5, 4))

for i in 1:5, j in 1:4
    placement_rates[order[i], order[j]] = est_mat_placement[i, j]
end
placement_rates

5×4 Matrix{Int64}:
  333   79   15    9
  346  211   55   20
  448  422  238   27
   51   76   48   54
 1044  917  584  179

The type allocation associated with these means is:

In [27]:
for j in 1:SELECT_COUNT
    println("TYPE ", j)
    println()
    i = 1
    for entry in est_alloc
        if order[entry] == j
            println(name_to_oid[institutions[i]], " ", institutions[i])
        end
        i += 1
    end
    println()
    println()
end

TYPE 1

914 Columbia University
1834 University of Pennsylvania
85 University of Wisconsin, Madison
37 Boston University
881 New York University
603 Cornell University
678 Northwestern University
499 Ohio State University
343 University of California, Berkeley
1691 London School of Economics and Political Science
658 University of Minnesota, Twin Cities
1926 Harvard University
36 University of California Los Angeles (UCLA)
336 Yale University
2282 Stanford University
40 University of Toronto
186 University of Michigan
24 Duke University
20 University of Maryland
886 Princeton University
394 University of Chicago


TYPE 2

110 Rice University
2010 Massachusetts Institute of Technology
253 University of Rochester
250 Vrije Universiteit Amsterdam
3203 London Business School
891 University of Bonn (Rheinische Friedrich-Wilhelms-Universität Bonn)
22 Syracuse University
218 University of Arizona
44 University of California, San Diego
362 Boston College
143 University of Warwick
203 Toulouse 

The numerical form of the type allocation, to be used for estimating the deep parameters, is:

In [20]:
print(est_alloc)

UInt8[0x04, 0x02, 0x03, 0x01, 0x03, 0x03, 0x02, 0x01, 0x03, 0x02, 0x03, 0x03, 0x02, 0x02, 0x02, 0x01, 0x03, 0x02, 0x02, 0x02, 0x01, 0x03, 0x03, 0x03, 0x02, 0x02, 0x02, 0x03, 0x01, 0x03, 0x02, 0x03, 0x04, 0x03, 0x02, 0x03, 0x03, 0x03, 0x03, 0x02, 0x03, 0x03, 0x02, 0x02, 0x03, 0x03, 0x02, 0x03, 0x02, 0x02, 0x03, 0x03, 0x03, 0x01, 0x04, 0x02, 0x01, 0x02, 0x01, 0x04, 0x01, 0x03, 0x04, 0x03, 0x02, 0x02, 0x01, 0x02, 0x01, 0x01, 0x03, 0x02, 0x02, 0x03, 0x03, 0x02, 0x02, 0x02, 0x03, 0x03, 0x01, 0x01, 0x04, 0x01, 0x02, 0x03, 0x02, 0x01, 0x03, 0x01, 0x03, 0x03, 0x03, 0x02, 0x02, 0x02, 0x03, 0x02, 0x03, 0x03, 0x02, 0x01, 0x02, 0x03, 0x03, 0x02, 0x02, 0x02, 0x03, 0x03, 0x02, 0x03, 0x04, 0x02, 0x02, 0x03, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x02, 0x03, 0x02, 0x03, 0x03, 0x02, 0x02, 0x03, 0x02, 0x02, 0x01, 0x04, 0x02, 0x01, 0x01, 0x04, 0x02, 0x02, 0x03, 0x04, 0x02, 0x03, 0x04, 0x01, 0x03, 0x02, 0x02, 0x02, 0x01, 0x03, 0x01, 0x02, 0x03, 0x03, 0x02, 0x03, 0x02, 0x02, 0x03, 0x01, 0x03, 0x03, 0x02, 0x02

In [25]:
new_alloc = [order[Int64(i)] for i in est_alloc];

In [32]:
open("type_allocation.json", "w") do f
    JSON.print(f, new_alloc)
end

## References

Karrer, B., and M. E. J. Newman (2011): "Stochastic Blockmodels and community structure in networks," Physical Review, 83(1).

Peixoto, T. (2014): "Efficient Monte Carlo and greedy heuristic for the inference of stochastic block models," Physical Review, 89(1).